In [150]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

In [140]:
! pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [141]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.utils.class_weight import compute_sample_weight

In [142]:
df = pd.read_csv('/content/drive/MyDrive/f1/qualifying.csv', index_col=0)

In [143]:
df.shape

(144, 41)

In [144]:
y = df_train['driver_id']
X = df_train.drop(['driver_id'], axis=1)

In [145]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [146]:
from sklearn import preprocessing
st_scaler = preprocessing.StandardScaler()
X_train = st_scaler.fit_transform(X_train)
X_test = st_scaler.transform(X_test)

In [151]:
params={'penalty': ['l1', 'l2'],
        'solver': ['saga', 'lbfgs'],
        'C': np.logspace(-3,1,20)}

logreg = LogisticRegression(class_weight='balanced', max_iter=10000)
logreg_cv = GridSearchCV(logreg, params, scoring='f1_micro')
logreg_cv.fit(X_train, y_train)
logreg_predictions = logreg_cv.best_estimator_.predict(X_test)

In [152]:
logreg_cv.best_score_

0.5626315789473685

In [154]:
f1_score(y_test, logreg_cv.best_estimator_.predict(X_test), average='micro')

0.14285714285714285

In [155]:
params={'criterion': ['gini', 'entropy'],
        'max_features': [0.8, 'auto', None],
        'max_depth': list(np.linspace(5, 55, 26)) + [None]}
        
rfc = RandomForestClassifier(class_weight='balanced_subsample')
rfc_cv = GridSearchCV(rfc, params, scoring='f1_micro')
rfc_cv.fit(X_train, y_train)
rfc_predictions = rfc_cv.best_estimator_.predict(X_test)

In [156]:
rfc_cv.best_score_

0.5526315789473685

In [157]:
f1_score(y_test, rfc_cv.best_estimator_.predict(X_test), average='micro')

0.4523809523809524

In [158]:
params={'gamma': np.logspace(-4, -1, 20),
        'C': np.logspace(-2, 1, 20),
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'decision_function_shape' : ['ovo', 'ovr']} 

svc = SVC(class_weight='balanced')
svc_cv = GridSearchCV(svc, params, scoring='f1_micro')
svc_cv.fit(X_train, y_train)
svc_predictions = svc_cv.best_estimator_.predict(X_test)

In [159]:
svc_cv.best_score_

0.4889473684210527

In [160]:
f1_score(y_test, svc_cv.best_estimator_.predict(X_test), average='micro')

0.47619047619047616

In [161]:
params = {'depth': [4,5,6,7,8,9, 10],
          'learning_rate' : [0.01,0.02,0.03,0.04],
          'iterations'    : [10, 20,30,40,50,60,70,80,90, 100]}

cbc = CatBoostClassifier(loss_function='MultiClass')
cbc_cv = GridSearchCV(cbc, params, scoring='f1_micro')
cbc_cv.fit(X_train, y_train)
cbc_cv_predictions = cbc_cv.best_estimator_.predict(X_test)

Выходные данные были обрезаны до нескольких последних строк (5000).
20:	learn: 1.7422852	total: 201ms	remaining: 564ms
21:	learn: 1.7320274	total: 203ms	remaining: 534ms
22:	learn: 1.7256383	total: 216ms	remaining: 534ms
23:	learn: 1.7188011	total: 219ms	remaining: 511ms
24:	learn: 1.7112053	total: 226ms	remaining: 496ms
25:	learn: 1.7033221	total: 229ms	remaining: 476ms
26:	learn: 1.6944075	total: 236ms	remaining: 464ms
27:	learn: 1.6774581	total: 237ms	remaining: 440ms
28:	learn: 1.6716775	total: 251ms	remaining: 442ms
29:	learn: 1.6635632	total: 265ms	remaining: 442ms
30:	learn: 1.6521722	total: 266ms	remaining: 421ms
31:	learn: 1.6457832	total: 268ms	remaining: 402ms
32:	learn: 1.6401182	total: 281ms	remaining: 400ms
33:	learn: 1.6349629	total: 303ms	remaining: 410ms
34:	learn: 1.6284238	total: 317ms	remaining: 408ms
35:	learn: 1.6228061	total: 330ms	remaining: 404ms
36:	learn: 1.6157467	total: 346ms	remaining: 402ms
37:	learn: 1.6057497	total: 347ms	remaining: 384ms
38:	learn: 1.5

In [162]:
cbc_cv.best_score_

0.5731578947368421

In [163]:
f1_score(y_test, cbc_cv.best_estimator_.predict(X_test), average='micro')

0.5

In [164]:
params = {
    'max_depth': [2,3,4,5,6,7,8,9,10],
    'n_estimators': range(60, 220, 40),
    'learning_rate': [0.1, 0.01, 0.05],
    'scale_pos_weight' : [1, 10, 25, 50, 75, 99, 100, 1000]}

sample_weights = compute_sample_weight(class_weight='balanced', y=y)

xgb = XGBClassifier(n_jobs = -1, objective='multi:softmax', sample_weight=sample_weights)
xgb_cv = GridSearchCV(xgb, params, scoring='f1_micro')
xgb_cv.fit(X_train, y_train)
xgb_predictions = xgb_cv.best_estimator_.predict(X_test)

In [165]:
xgb_cv.best_score_

0.5731578947368421

In [166]:
f1_score(y_test, xgb_cv.best_estimator_.predict(X_test), average='micro')

0.5476190476190477

In [167]:
xgb_cv.best_params_


{'learning_rate': 0.01,
 'max_depth': 2,
 'n_estimators': 100,
 'scale_pos_weight': 1}

In [168]:
sample_weights = compute_sample_weight(class_weight='balanced', y=y)

xgb_model = XGBClassifier(n_jobs = -1, objective='multi:softmax', sample_weight=sample_weights, learning_rate=0.01,
                    max_depth=2, n_estimators=100, scale_pos_weight=1)
xgb_model.fit(X, y)

XGBClassifier(learning_rate=0.01, max_depth=2, n_jobs=-1,
              objective='multi:softprob',
              sample_weight=array([ 0.20444444,  0.20444444,  0.20444444,  0.20444444,  0.20444444,
        0.20444444,  0.20444444,  0.20444444,  0.20444444,  0.20444444,
        0.20444444,  0.20444444,  0.20444444,  0.20444444,  0.20444444,
        0.20444444,  0.20444444,  0.20444444,  1.27777778,  0.20444444,
        0.20444444,  0.20444444,  5.111...
       15.33333333,  0.20444444,  0.80701754,  1.17948718,  1.17948718,
        0.20444444,  0.80701754,  0.20444444,  1.17948718,  1.17948718,
        1.17948718,  1.17948718,  1.17948718,  0.20444444,  0.20444444,
        1.17948718,  1.17948718,  0.80701754, 15.33333333,  0.20444444,
        1.17948718,  0.80701754,  0.20444444,  0.20444444,  0.20444444,
        1.17948718,  1.17948718, 15.33333333,  1.17948718,  1.17948718,
        1.17948718,  1.17948718,  1.17948718]))

In [169]:
filename = 'model.sav'
pickle.dump(xgb_model, open(filename, 'wb'))